In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 207kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 26.0MB/s]


In [4]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
model = NeuralNetwork().to(device)
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.176964  [   64/60000]
loss: 2.170758  [ 6464/60000]
loss: 2.116460  [12864/60000]
loss: 2.131694  [19264/60000]
loss: 2.090571  [25664/60000]
loss: 2.030125  [32064/60000]
loss: 2.066307  [38464/60000]
loss: 1.986134  [44864/60000]
loss: 1.980405  [51264/60000]
loss: 1.933152  [57664/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 1.917345 

Epoch 2
-------------------------------
loss: 1.943442  [   64/60000]
loss: 1.916196  [ 6464/60000]
loss: 1.801460  [12864/60000]
loss: 1.845287  [19264/60000]
loss: 1.743946  [25664/60000]
loss: 1.690023  [32064/60000]
loss: 1.724895  [38464/60000]
loss: 1.617375  [44864/60000]
loss: 1.635240  [51264/60000]
loss: 1.553608  [57664/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.555600 

Epoch 3
-------------------------------
loss: 1.615651  [   64/60000]
loss: 1.585171  [ 6464/60000]
loss: 1.436360  [12864/60000]
loss: 1.508507  [19264/60000]
loss: 1.401397  [25664/60000]
loss: 1.384436  [32064/600